#Machine Learning Estadístico para Interfaces Cerebro-Computadora

## Trabajo Práctico de Laboratorio de Computación II - Parte III: Filtrado espacial para extracción de características


❗Antes de comenzar recordá hacer una copia de este documento de manera que puedas editarlo y guardar los cambios en tu Drive.

Este TPLC tiene como objetivo que el alumno sea capaz de:
1. Afianzar los conocimientos sobre filtrado espacial en el contexto de extracción de características
2. Utilizar y comprender el uso de funciones ya implementadas para extraer componentes de máxima discriminabilidad o explicabilidad de la variable objetivo
3. Comprender la diferencia de aplicar un método para reducción de dimensionalidad o extracción de características


Es **requisito** para aprobar este curso que esta guía práctica sea completada y enviada para su evaluación. Se aceptan trabajos realizados en forma grupal de *hasta dos alumnos por grupo*.

In [1]:
# Importamos las bibliotecas y funciones
import numpy as np
import matplotlib.pyplot as plt
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, pick_types, events_from_annotations
import mne

# Filtrado espacial para extracción de características

## Patrones Espaciales Comunes (CSP)

El método de Patrones Espaciales Comunes (CSP) es una técnica de filtrado espacial bien conocida originalmente pensada para problemas de clasificación binaria. Proyecta los datos en un nuevo subespacio en el cual la varianza de una clase se maximiza mientras se minimiza para la otra clase [1]. Dado que la varianza de una señal filtrada en una banda frecuencial es igual a la potencia de banda, es posible extraer características espaciales y de frecuencia específica que permitan luego alimentar un clasificador de aprendizaje automático.

En esta práctica nos vamos a centrar en la comprensión de CSP como técnica de reducción de dimensionalidad y extractor de características en el que se maximiza/minimiza la varianza.

## Los datos

Vamos a utilizar la base de datos de imaginería motora de EEGBCI que utilizamos antes, la cual fue originalmente publicada en [2] y consiste en varios experimentos de BCI realizados a la misma persona. En particular en esta Colab vamos utilizar los datos del sujeto 1 durante las rondas correspondientes a la imaginación del movimiento de pies vs manos

In [2]:
from mne.datasets import eegbci
event_id = dict(hands=2, feet=3)  # MI: hands vs feet runs
# for a given subject
subject = 1

runs = [6, 10, 14]  # motor imagery: hands vs feet runs

raw_fnames = eegbci.load_data(subject, runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
eegbci.standardize(raw)  # set channel names
montage = make_standard_montage("standard_1005")
raw.set_montage(montage)

Using default location ~/mne_data for EEGBCI...


Download complete in 03m22s (7.4 MB)
Extracting EDF parameters from C:\Users\felip\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\felip\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\felip\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<RawEDF | S001R06.edf, 64 x 60000 (375.0 s), ~29.4 MB, data loaded>

CSP es una técnica supervisada que se basa en la matriz de covarianza promedio entre las pruebas pertenecientes a cada una de las clases. Por lo tanto, primero necesitamos dividir nuestros datos en segmentos (pruebas) de interés. Aquí, vamos a dividir los datos en segmentos de 2 segundos de duración a partir de 0.5 segundos después del inicio de la señal.

In [3]:
# before epoching, apply a filter!
# Apply band-pass filter to cover the alpha and beta band
raw.filter(8.0, 30.0, fir_design="firwin", skip_by_annotation="edge")
# select the epoch time
tmin, tmax = 0.5, 2.5
events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))
# T1:hands, T2:feet

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude="bads")
# get the epoch object
epochs = Epochs(
    raw,
    events,
    event_id,
    tmin,
    tmax,
    proj=True,
    picks=picks,
    baseline=None,
    preload=True,
)

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 321 original time points ...
0 bad epochs dropped


❓ ¿Por qué elegimos esa ventana de análisis? ¿Por qué filtramos la señal entre 8 y 30 Hz?

### <span style='color:#FF0000'>Respuesta:</span>


Como método supervisado de extracción de características, es muy importante contar con las etiquetas de cada segmento de EEG a analizar. En lo que sigue extraemos los datos y etiquetas del objeto `epochs`

In [5]:
# Get data and labels
labels = epochs.events[:, -1]
data = epochs.get_data()

print(data.shape)
print(labels.shape)
print("Hay %d trials de los cuales %d pertenencen a la clase MI de manos." %(len(labels), abs(np.sum(labels==2))))
print(labels)

(45, 64, 321)
(45,)
Hay 45 trials de los cuales 21 pertenencen a la clase MI de manos.
[3 2 2 3 2 3 3 2 2 3 3 2 2 3 3 2 3 3 2 3 2 2 3 3 2 2 3 3 2 3 3 2 3 2 2 3 2
 3 2 3 3 2 3 2 3]


### Implementando CSP

Estudiamos la clase [CSP](https://mne.tools/stable/generated/mne.decoding.CSP.html) de MNE 👇

In [31]:
# import from mne
from mne.decoding import CSP

In [32]:
vars(CSP())

{'n_components': 4,
 'rank': None,
 'reg': None,
 'cov_est': 'concat',
 'transform_into': 'average_power',
 'log': None,
 'norm_trace': False,
 'cov_method_params': None,
 'component_order': 'mutual_info'}

❓
1. El parámetro `n_components`, ¿qué define dentro del método?
2. ¿Qué opciones existen para el parámetro `transform_into`? ¿Qué efecto tiene en la implementación y uso del método?
3. ¿Qué alternativas existen para el parámetro `component_order`? ¿Qué cambiará en la implementación del método si elijo una u otra opción?

🤓 Escribí acá tus rtas

1...

2...

3...


#### Manos a la obra 🤝

1. Crea un objeto `CSP` que transforme los datos en el espacio de los componentes de CSP y reduzca la dimensionalidad del problema a 6. Como método de ordenamiento utilzá `alternate`

In [33]:
CSP1 = CSP(n_components=6, component_order='alternate', transform_into='average_power')
CSP2 = CSP(n_components=6, component_order='alternate', transform_into='csp_space')

Sabemos que esto va más allá del módulo actual 🙄, pero CSP es un extractor de características supervisado basado en datos. Como tal, define un método de aprendizaje automático del que el "extractor" necesita ser "aprendido" y luego aplicado en nuevos datos no vistos. Por esta razón, necesitamos dividir nuestros datos en dos conjuntos: conjunto de entrenamiento y conjunto de prueba. Para eso vamos a utilizar la librería [sklearn](https://scikit-learn.org/)

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

Genial, ahora que ya tenemos los datos dividos en dos subconjuntos separados, seguimos...


2. Ajustá el objeto `csp` con los datos de entrenamiento y devolvé en pantallá el tamaño de la matriz de demezcla Wcsp. Responde: ¿tiene sentido el tamaño de esta matriz?

💡 TIP: utiliza el método `.fit` para ajustar el modelo. Una vez ajutado el modelo podes acceder a los filtros espaciales mediante `.filters_`

In [ ]:
# escribí acá tú código

In [ ]:
# escribí acá tú código

3. Explorá el poder de discriminabilidad de cada componente. ¿Sos capaz de observar alguna información discriminativa que diference imaginación de pies vs imaginación de manos?

💡 TIP: podes usar el método plot_patterns de csp para graficar los patrones espaciales

In [ ]:
# escribí acá tú código

4. Obtené las componentes csp para los datos de testeo (`X_test_csp`). ¿Cuál es el tamaño de `X_test_csp`? ¿Por qué?

In [ ]:
# escribí acá tú código

Para terminar de comprender el método CSP. Vamos a graficar los trazos temporales de los primeros 6 componentes de CSP:

In [ ]:
n_components = 6

nt, nc, ns = np.shape(X_test_csp)

time = np.arange(0, ns/raw.info["sfreq"], 1/raw.info["sfreq"])

names_components = ['CSP0', 'CSP1', 'CSP2', 'CSP3', 'CSP4', 'CSP5']
# para los primeros 5 trials..
for tt in range(5):
  plt.figure()
  if y_test[tt] == 3:
    class_name = 'Hands'
  else:
    class_name = 'Feet'
  plt.title("CSP components  trial " + str(tt) + " class " + class_name)
  for i in range(n_components):
    plt.plot(time, 4*i+X_test_csp[tt, i, :])
  plt.yticks(np.arange(0, 4*n_components, 4), names_components)
  plt.ylabel("Component")
  plt.xlabel("Time (s)")

❓ ¿Qué observas? ¿Qué es lo que cambia para los componentes entre trials de diferente clase?

👀 Una ayudita: prestá atención al componente CSP1 y CSP3 entre los trials de mano y pies.

....

❔ ¿ Habrá alguna forma de cuantificar eso que observamos?

4. Calculá las features de csp (`csp_features_var`) a sabiendas que es posible calcular la potencia de una señal filtrada mediante el cálculo de la varianza de dicha señal filtrada. Luego verificá si las features extraídas son similares al aplicar la fórmula de potencia media para una señal causal:

 $P = \frac{1}{N}∑_{n=0}^{∞}|x[n]| ^2$

 Llama a estas features `csp_features`.

In [ ]:
# escribí acá tú código

In [ ]:
# escribí acá tú código

Para seguir comprendiendo CSP, vamos a considerar que los primeros 2 componentes son lo más discriminativos. Por lo tanto, podemos graficar en el plano cómo las features de una clase y de la otra se agrupan:

In [ ]:
plt.scatter(csp_features[y_test==2, 0], csp_features[y_test==2,1], s=40, color='blue', label='Hands MI')
plt.scatter(csp_features[y_test==3, 0], csp_features[y_test==3,1], s=40, color='red', label = 'Feet MI')
plt.xlabel('CSP feature 0')
plt.ylabel('CSP feature 1')
plt.legend()

❓ ¿Qué ocurre si graficamos CSP feature 0 vs CSP feauture 4? ¿Por qué? ¿Era esperado?

In [ ]:
#probá acá!

## Para seguir pensando....

❓


1. ¿Qué hubiese ocurrido si en vez de utilizar `alternate` como método de ordenamiento habríamos usado `Mutual information`? Probalo y fijate como cambian los topomaps!
2. ¿Puedo inferir el número óptimo de features a extraer en función de la observación de los mapas topográficos de los patrones espaciales de CSP?
3. ¿Están ordenados los componentes?
4. A mayor cantidad de trial, ¿qué mejorará en el método?



1 ...

2 ...

3 ...

# References
[1] Blankertz, B., Tomioka, R., Lemm, S., Kawanabe, M., & Muller, K. R. (2007). Optimizing spatial filters for robust EEG single-trial analysis. IEEE Signal processing magazine, 25(1), 41-56.

[2] Gerwin Schalk, Dennis J. McFarland, Thilo Hinterberger, Niels Birbaumer, and Jonathan R. Wolpaw. BCI2000: a general-purpose brain-computer interface (BCI) system. IEEE Transactions on Biomedical Engineering, 51(6):1034–1043, 2004. doi:10.1109/TBME.2004.827072.
